In [3]:

!jupyter nbconvert --to html --TemplateExporter.exclude_code_cell=True --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True scc2425-lab5.ipynb 2> /dev/null
!jupyter nbconvert --to slides --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True scc2425-lab5.ipynb 2> /dev/null

# Cloud Computing Systems
## 2024/25

Lab 5
https://smduarte.github.io/scc2425/

Sérgio Duarte, Kevin Gallagher 

# Goals

+ Create Azure Functions in Java and deploy them in the Azure Cloud Platform


# Azure Functions

### Brochure selling points

Azure Functions is a **serverless** solution that allows you to write less code, 
**maintain less infrastructure**, and save on costs. 

Instead of worrying about deploying and maintaining servers, the cloud infrastructure 
***provides all the up-to-date resources*** needed to keep your applications running.

You focus on the code that matters most to you, in the most productive language for you, and Azure Functions handles the rest.

# Documentation


### Azure documentation on writing functions in Java:

[https://docs.microsoft.com/en-us/azure/azure-functions/functions-reference-java](https://docs.microsoft.com/en-us/azure/azure-functions/functions-reference-java)

### Triggers and bindings:

#### Timer
[https://docs.microsoft.com/en-us/azure/azure-functions/functions-bindings-timer?tabs=java](https://docs.microsoft.com/en-us/azure/azure-functions/functions-bindings-timer?tabs=java)

#### HTTP
[https://docs.microsoft.com/en-us/azure/azure-functions/functions-bindings-http-webhook](https://docs.microsoft.com/en-us/azure/azure-functions/functions-bindings-http-webhook)

#### Blob storage
[https://docs.microsoft.com/en-us/azure/azure-functions/functions-bindings-storage-blob](https://docs.microsoft.com/en-us/azure/azure-functions/functions-bindings-storage-blob)

#### CosmosDB
[https://docs.microsoft.com/en-us/azure/azure-functions/functions-bindings-cosmosdb-v2](https://docs.microsoft.com/en-us/azure/azure-functions/functions-bindings-cosmosdb-v2)

# Create a project for the functions

## Alternative #1

Use the sample project provided: [scc2425-lab5-code.zip](scc2425-lab5-code.zip).


## Alternative #2

Execute the command below on an empty directory:

`mvn archetype:generate -DarchetypeGroupId=com.microsoft.azure -DarchetypeArtifactId=azure-functions-archetype`


# Configuration of pom.xml

The properties below need to be updated...
```xml
		<functionAppName>funxxxxxxnortheurope</functionAppName>
		<functionRegion>northeurope</functionRegion>
		<functionStorageAccountName>stoxxxxxxnortheurope</functionStorageAccountName>
		<functionResourceGroup>rgxxxxxx-northeurope</functionResourceGroup>
```

Pay attention to how the various resources are named. They need to be adapted
to match your deployment. Replace xxxxxx with your student id.

The resources names above follow the convention that is implemented in the project 
that will help with automatic deployment of multiple resources: [scc2425-mgt-code.zip](scc2425-mgt-code.zip).

# Azure Functions Examples

#### HttpTrigger

+ React to HTTP requests that match a given pattern. 

#### TimerTrigger

+ Implement periodic actions.

####  BlobTrigger

+ React to Blob Storage uploads with filenames that match a given pattern. 

#### CosmosDBTrigger

+ Reach to CosmosDB container updates


# Http Function Example
```java
public class HttpFunction {
	private static final String TEXT = "text";
	private static final String HTTP_TRIGGER_NAME="req";
	private static final String HTTP_FUNCTION_NAME="HttpExample";
	private static final String HTTP_TRIGGER_ROUTE="serverless/echo/{" + TEXT + "}";
	
	@FunctionName(HTTP_FUNCTION_NAME)
    public HttpResponseMessage run(
            @HttpTrigger(
                name = HTTP_TRIGGER_NAME,
                methods = {HttpMethod.GET, HttpMethod.POST},
                authLevel = AuthorizationLevel.ANONYMOUS,
                route = HTTP_TRIGGER_ROUTE)
                HttpRequestMessage<Optional<String>> request,
                @BindingName(TEXT) String text,
            final ExecutionContext context) {
        context.getLogger().info("Java HTTP trigger processed a request.");
        return request.createResponseBuilder(HttpStatus.OK).body("Hello, " + text).build();
    }
}
```

**Documentation:**

[https://docs.microsoft.com/en-us/azure/azure-functions/functions-bindings-http-webhook-trigger?tabs=java](https://docs.microsoft.com/en-us/azure/azure-functions/functions-bindings-http-webhook-trigger?tabs=java)

**Notes:**

The `route` parameter defines the path that triggers the execution of the function. 

It is possible to capture variables/parameters, by enclosing their name in `{}`, and in conjunction with the `@BindingName` annotation.

# Time Function Example

```java
public class TimerFunction {
  private static final String TIMER_FUNCTION_NAME = "timerFunctionExample";
  private static final String TIMER_TRIGGER_NAME = "timerFunctionTrigger";
  private static final String TIMER_TRIGGER_SCHEDULE = "0 * * * * *";
	
  @FunctionName(TIMER_FUNCTION_NAME)
  public void run( @TimerTrigger(name = TIMER_TRIGGER_NAME, schedule = TIMER_TRIGGER_SCHEDULE) 
                  String timerInfo, 
                  ExecutionContext context) {
	 
      context.getLogger().info("Timer is triggered: " + timerInfo);
  }
}
```

**Documentation:**

[https://docs.microsoft.com/en-us/azure/azure-functions/functions-bindings-timer?tabs=java](https://docs.microsoft.com/en-us/azure/azure-functions/functions-bindings-timer?tabs=java)

**Notes:** 

The *schedule* determines how often the trigger executes. It uses the [crontab](https://en.wikipedia.org/wiki/Cron) format.

`0 * * * * *` - means execute at the 0 second of every minute, hour, day, ... 

# BlobStorage Function Example
```java
public class BlobStoreFunction {
	private static final String NAME = "name";
	private static final String PATH = "shorts/{" + NAME + "}";
	private static final String BLOBS_TRIGGER_NAME = "blobFunctionTrigger";
	private static final String BLOBS_FUNCTION_NAME = "blobFunctionExample";
	private static final String DATA_TYPE = "binary";
	private static final String BLOBSTORE_CONNECTION_ENV = "BlobStoreConnection";

	@FunctionName(BLOBS_FUNCTION_NAME)
	public void blobFunctionExample(
			@BlobTrigger(name = BLOBS_TRIGGER_NAME, 
			dataType = DATA_TYPE, path = PATH, 
			connection = BLOBSTORE_CONNECTION_ENV) byte[] content,
			@BindingName("name") String blobname, ExecutionContext context) {

		context.getLogger().info(String.format("blob : %s, uploaded with %d bytes", blobname, content.length));
	}
}
```

**Documentation:**

[https://docs.microsoft.com/en-us/azure/azure-functions/functions-bindings-storage-blob-trigger?tabs=java](https://docs.microsoft.com/en-us/azure/azure-functions/functions-bindings-storage-blob-trigger?tabs=java)

**Notes**: 

The `@BlobTrigger` annotation includes the `connection` argument that needs to contain the name of `environment` variable/settings that stores the connection string to the blob storage. In other words,
`connection` does not contain the actual connection string.

In the example above, `BlobStoreConnection` is the environment variable that is set via the `az functionapp config appsettings set ...` command as show in the "Advanced/Automated Deployment" section next.

# CosmosDB Function Example

```java
public class CosmosDBFunction {
    private static final String COSMOSDB_CONNECTION_ENV="AzureCosmosDBConnection";
	...
		
    @FunctionName(COSMOSDB_FUNCTION_NAME)
    public void run(@CosmosDBTrigger(name = COSMOSDB_TRIGGER_NAME,
    					databaseName = COSMOSDB_DATABASE_NAME,
    					connection = COSMOSDB_CONNECTION_ENV, 
    					containerName = COSMOSDB_CONTAINER_NAME,
    					leaseContainerName = COSMOSDB_LEASES_CONTAINER,
    					createLeaseContainerIfNotExists = true)
        				String usersJson,
        				ExecutionContext context ) {
    	
    	var list = GSON.decode(usersJson, new TypeToken<List<User>>() {});
    	for( var u : list )
    		context.getLogger().info("Added user: " + u);
    }
}
```

**Documentation:**

[https://docs.microsoft.com/en-us/azure/azure-functions/functions-bindings-cosmosdb-v2-trigger?tabs=java](https://docs.microsoft.com/en-us/azure/azure-functions/functions-bindings-cosmosdb-v2-trigger?tabs=java)

**Notes:** 

The trigger receives the list of updates in the form of a JSON string that needs to be decoded. 

**Only triggers for inserts or updates, not deletes**.

# Connection Strings Environment Variables

The **CosmosDB Function** and the **BlobStorage Function** examples, above, need connection strings to access their respective resources:

`AzureCosmosDBConnection` and `BlobStoreConnection`

These environment variables are set using the azure cli commands, like so:

```
    az functionapp config appsettings set
      --name <functions-name> \
      --resource-group <functions-group> \
      --settings "BlobStoreConnection=<actual connection string>"
``` 

**Notes:**

The project that does the automatic deployment of resources to Azure [scc2425-mgt-code.zip](scc2425-mgt-code.zip) generates scripts (files ending in .sh) that contain that commands that need to be executed.

In a Linux/MacOS client, after the deployment completes, simply execute the script with: `sh azureprops-westeurope.sh`, replacing the region with the one that was actually used. In Windows, this is one
more reason to use [Linux WSL](https://learn.microsoft.com/en-us/windows/wsl/install)...

# Debugging Azure Functions 

In the Azure portal, it is possible to inspect the Azure functions that are deployed.

<img src="functions-1.png" width="100%"></img>

# Debug Azure Functions : Invocations (1)

<img src="functions-2.png" width="100%"></img>

# Debug Azure Functions : Invocations (2)

<img src="functions-3.png" width="100%"></img>

**Notes**

The list of invocations updates with a delay of several seconds...

# Debug Azure Functions : Logs

<img src="functions-4.png" width="100%"></img>

**Notes**

Connecting to Logs takes a while and only shows executions that triggered after the connection was established...

# Code provided

The code provided [scc2425-lab5-code.zip](scc2425-lab5-code.zip) is a Maven project with example Azure functions that can be deployed to Azure. 

You will need to change the properties in pom.xml – check slide 5. 

For compiling and deploying, just run:

mvn clean compile package azure-functions:deploy

After deploying, do not forget to set the properties used by the Azure functions. You can use the .sh script file created by executing `scc.mgt.AzureManagement` from [scc2425-mgt-code.zip](scc2425-mgt-code.zip) – make sure the name of the function app and the resource group are correct. 

**Notes:**

1. Edit MY_ID in class `AzureManagement.java` and maybe choose a different the deployment region or regions.
2. Generate the jar: `mvn clean compile assembly:single`
3. Execute the deployment: `java -cp target/scc2425-mgt-1.0-jar-with-dependencies.jar scc.mgt.AzureManagement`
4. Execute the generated `azureprops-<region>.sh` script to set the environment variables.  